In [23]:
import pandas as pd 
import numpy as np
select_blackbox = ['dt', 'nn', 'rf'][1]
sanity = True
if(sanity):
    df = pd.read_csv("data/output/sanity_result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
    df['query'].fillna('true', inplace=True)

else:
    df = pd.read_csv("data/output/result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
    df = df[df['blackbox'] == select_blackbox]


print("minimum iterations:", min(df.groupby(['query'])['explanation'].count()), "max iterations:",max(df.groupby(['query'])['explanation'].count()))
df.head()

minimum iterations: 47 max iterations: 100


,accuracy,blackbox,dataset,explainer,explanation,explanation size,positive counterexamples,query,random words checked,syntactic grammar,terminate,time,time learner,time verifier,total counterexamples
0,1.0,decision tree,zoo,sygus,(and (not breathes) fins),2.0,0.500000,true,962.0,1.0,1.0,0.990528,0.334000,0.655108,6.0
1,1.0,decision tree,zoo,sygus,(and (not breathes) fins),2.0,0.428571,true,964.0,1.0,1.0,0.987706,0.325856,0.660411,7.0
2,1.0,decision tree,zoo,sygus,(and (not breathes) fins),2.0,0.333333,true,1030.0,1.0,1.0,1.183521,0.432247,0.749661,9.0
3,1.0,decision tree,zoo,sygus,(and (not breathes) fins),2.0,0.333333,true,969.0,1.0,1.0,1.020707,0.392725,0.626534,9.0
4,1.0,decision tree,zoo,sygus,(and (not breathes) fins),2.0,0.250000,true,827.0,1.0,1.0,0.630564,0.246163,0.383300,8.0


In [24]:
# take median
group_list = ['dataset','explainer','query', 'syntactic grammar']
df_med = df.groupby(group_list).mean()
df_med.reset_index(inplace=True)
df_med

,dataset,explainer,query,syntactic grammar,accuracy,explanation size,positive counterexamples,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,zoo,sygus,breathes = 0,1.0,0.890909,0.77,0.017951,603.23,1.0,0.172746,0.066136,0.106296,27.79
1,zoo,sygus,breathes = 1,1.0,0.977273,0.68,0.016543,584.09,1.0,0.151084,0.057888,0.092929,24.49
2,zoo,sygus,fins = 0,1.0,0.986532,0.72,0.014220,588.73,1.0,0.157305,0.059228,0.097778,30.34
3,zoo,sygus,milk = 1,1.0,1.000000,0.70,0.013073,588.46,1.0,0.164348,0.065890,0.098176,21.90
4,zoo,sygus,true,1.0,1.000000,2.00,0.054862,722.42,1.0,0.563055,0.302349,0.259896,52.14


In [25]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['dataset','explainer','query', 'syntactic grammar'], as_index = False):
    item.reset_index(inplace=True, drop=True)
    if(len(item) == 1):
        query_explanations[key] = item['explanation'].item()
    else:
        # nearest explanation to the average accuracy
        explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['dataset'] == key[0]) & (df_med['explainer'] == key[1]) & (df_med['syntactic grammar'] == key[3]) & (df_med['query'] == key[2])].item()).abs().argsort()][0]]['explanation']
        query_explanations[key] = explanation

In [26]:
df_output = df_med
df_output = df_output.drop("terminate", axis=1)
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][(df_output['dataset'] == key[0]) & (df_output['explainer'] == key[1]) & (df_output['syntactic grammar'] == key[3]) & (df_output['query'] == key[2])] = query_explanations[key]

# reorganise columns
df_output = df_output[[ 'dataset',  'syntactic grammar', 'explainer','query', 'explanation', 'accuracy', 'time', 'time learner', 'time verifier', 'random words checked', 'total counterexamples', 'positive counterexamples', 'explanation size']]
df_output = df_output.round(2)
df_output.head(n=10)

,dataset,syntactic grammar,explainer,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples,explanation size
0,zoo,1.0,sygus,breathes = 0,NaN,0.89,0.17,0.07,0.11,603.23,27.79,0.02,0.77
1,zoo,1.0,sygus,breathes = 1,false,0.98,0.15,0.06,0.09,584.09,24.49,0.02,0.68
2,zoo,1.0,sygus,fins = 0,NaN,0.99,0.16,0.06,0.10,588.73,30.34,0.01,0.72
3,zoo,1.0,sygus,milk = 1,NaN,1.00,0.16,0.07,0.10,588.46,21.90,0.01,0.70
4,zoo,1.0,sygus,true,(and (not breathes) fins),1.00,0.56,0.30,0.26,722.42,52.14,0.05,2.00


In [27]:
import matplotlib.pyplot as plt
import pickle
from IPython.display import Markdown, display
from trustable_explanation import helper_functions
import numpy as np


features = ['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize']
# print results in an explainable manner
for key, item in df_output.groupby(['dataset','query','explainer','syntactic grammar'], as_index = False):
    print("\n\n")
    display(Markdown("### Query"))
    for i in key:
        print(i)
    if(key[3] == 0):
        continue
    display(Markdown("### Result"))
    if(key[1] == "logistic regression"):
        feature_importance = np.fromstring(item['explanation'].item()[1:-1], dtype=np.float, sep=' ')
        feature_importance = 100.0 * (feature_importance / (abs(feature_importance).max()))
        sorted_idx = np.argsort(abs(feature_importance))
        pos = np.arange(sorted_idx.shape[0]) + .5
        featfig = plt.figure()
        featax = featfig.add_subplot(1, 1, 1)
        featax.barh(pos, feature_importance[sorted_idx], align='center')
        featax.set_yticks(pos)
        featax.set_yticklabels(np.array(features)[sorted_idx])
        featax.set_xlabel('Relative Feature Importance')
        plt.tight_layout()   
        plt.show()
    elif(key[1] == "decision tree"):
        dt = None
        with open(item['explanation'].item(), 'rb') as fid:
            dt = pickle.load(fid)
        print(helper_functions.tree_to_code(dt,features))
    else:
        print(item['explanation'].item())
    print("\n\naccuracy",item['accuracy'].item())


### Query

zoo
breathes = 0
sygus
1.0


### Result

nan


accuracy 0.89





### Query

...

nan


accuracy 1.0





### Query

zoo
true
sygus
1.0


### Result

(and (not breathes) fins)


accuracy 1.0


In [28]:
# to remote 
# df_output.to_csv("data/output/summary.csv", index = False)
# df_output.to_csv("/home/bishwamittra/Dropbox/trustable_explanations/result.csv", index = False)

In [29]:
def time_format(x):
    if(x > 300):
        return '300'
    else:
        return '%.2f' % x
def dataset_format(x):
    dic = {
        'adult' : 'Adult',
        'zoo' : 'Zoo',
        'iris' : 'Iris'
    }
    return dic[x]
def percentage(x):
    return '%.2f' % x
def single_decimal(x):
    return '%.1f' % x

def integer_format(x):
    return '%.0f' % x

if(sanity):
    print(df_output[df_output['syntactic grammar'] == 1][['query', 'explanation', 'explanation size', 'accuracy', 'time']].to_latex(index=False, header=['Query', 'Explanation', 'Size', 'Accuracy', 'Time'], formatters={'time': time_format, 'accuracy': single_decimal, 'explanation size':single_decimal}))

\begin{tabular}{llrrr}
\toprule
        Query &                 Explanation & Size & Accuracy & Time \\
\midrule
 breathes = 0 &                         NaN &  0.8 &      0.9 & 0.17 \\
 breathes = 1 &                       false &  0.7 &      1.0 & 0.15 \\
     fins = 0 &                         NaN &  0.7 &      1.0 & 0.16 \\
     milk = 1 &                         NaN &  0.7 &      1.0 & 0.16 \\
         true &   (and (not breathes) fins) &  2.0 &      1.0 & 0.56 \\
\bottomrule
\end{tabular}



In [49]:


df_output['learner(%)'] = df_output['time learner']/df_output['time']
df_output['verifier(%)'] = df_output['time verifier']/df_output['time']
print(df_output[df_output['syntactic grammar'] == 1][['dataset', 'explanation size', 'accuracy', 'time', 'learner(%)', 'verifier(%)', 'random words checked']].to_latex(index=False, formatters=[dataset_format, , percentage, time_format, percentage, percentage, integer_format], header = ['Dataset', 'Explanation size', 'Accuracy', 'Time', 'Learner(%)', 'Verifier(%)', 'Test inputs']))

\begin{tabular}{lrrrrrr}
\toprule
Dataset & Explanation size & Accuracy & Time & Learner(\%) & Verifier(\%) & Test inputs \\
\midrule
  Adult &             4.00 &     0.69 &  300 &       0.88 &        0.12 &        4763 \\
   Iris &             8.00 &     0.80 &  300 &       0.98 &        0.01 &         112 \\
    Zoo &             4.00 &     0.91 &  300 &       0.96 &        0.04 &        6782 \\
\bottomrule
\end{tabular}



In [31]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
fontsize = 22
labelsize = 18
df_output

,dataset,syntactic grammar,explainer,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples,explanation size,learner(%),verifier(%)
0,adult,0.0,sygus,"- threshold: 0.1\n- specific_input: [1.0, 0.24...",(and sex race_4 (= age (/ 1232876712328767 50...,0.54,1.23,0.20,1.02,89.0,17.0,0.06,11.0,0.162602,0.829268
1,adult,1.0,sygus,"- threshold: 0.1\n- specific_input: [1.0, 0.24...",(and (not race_2) (and (not race_3) (and (< e...,0.69,301.05,264.12,35.99,4763.0,28.5,0.06,4.0,0.877329,0.119548
2,iris,0.0,sygus,- threshold: 0.1\n- specific_input: [0.3888888...,(let ((_let_0 (= petal-width (/ 279 500)))) (...,0.80,302.49,44.02,257.67,102613.0,132.0,0.89,604.5,0.145525,0.851830
3,iris,1.0,sygus,- threshold: 0.1\n- specific_input: [0.3888888...,(or (and (> sepal-length (/ 1 2)) (< petal-wi...,0.80,301.11,296.09,4.02,111.5,14.0,0.36,8.0,0.983328,0.013351
4,zoo,0.0,sygus,"- threshold: 0.1\n- specific_input: [1.0, 0.0,...",(and hair milk airborne toothed backbone brea...,1.00,0.51,0.25,0.27,89.0,17.0,0.06,16.0,0.490196,0.529412
5,zoo,1.0,sygus,"- threshold: 0.1\n- specific_input: [1.0, 0.0,...",(and (not feathers) (and (not eggs) (and (not...,0.91,301.04,288.87,11.17,6782.5,33.5,0.05,4.0,0.959573,0.037105


In [47]:
save = True
group_list = ['model']

os.system("mkdir data/output/fig")

sns.set(rc={'figure.figsize':(10,8)})
# sns.set_style("whitegrid", {'axes.grid' : False})

data = df_output.copy()
data['syntactic grammar'] = data['syntactic grammar'].map({1.0: 'syn', 0.0: 'no syn'})
data['dataset'] = data['dataset'].map({
        'adult' : 'Adult',
        'zoo' : 'Zoo',
        'iris' : 'Iris'
    })
sns.catplot( x = 'dataset', y = 'accuracy', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Accuracy', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='best', fontsize=labelsize-4, frameon=False)
# plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/accuracy.pdf")
    pass
else:
    plt.show()
plt.clf()


sns.catplot( x = 'dataset', y = 'time', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Time', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='best', fontsize=labelsize-4, frameon=False)
plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/time.pdf")
    pass
else:
    plt.show()
plt.clf()

sns.catplot( x = 'dataset', y = 'explanation size', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Explanation size', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='best', fontsize=labelsize-4, frameon=False)
plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/size.pdf")
    pass
else:
    plt.show()
plt.clf()



<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>